# Hyperparameter tuning with Optuna and Pytorch

This notebook shows the hyperparameter optimization process of LSTM neural network, constructed with Pytorch. The optimization process was done using Optuna library. The LSTM neural network takes stock price time-series data as multi-time-step input and output prediction for 1 time-step. Time-series data was downloaded using Alpha Vantage API. 

In [1]:
# Load data


import pandas as pd
import plotly.express as px
from torch.nn import *
from sklearn.metrics import r2_score
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import plotly.graph_objects as go
import datetime
from functools import partial
import numpy as np
import os
import torch
import optuna
from torch import nn
#Specify column labels
label = 'close'
features = 'close'

#Normalize function
def normalize(df, max_df, min_df,colname): 
    normed = [(i - min_df)/(max_df - min_df) for i in df]
    normed_df = pd.DataFrame(normed)
    normed_df.columns = [colname]
    return normed_df

#Function to reverse normalization
def denormalize(df, max_df, min_df,colname):
    denormed= [i*(max_df -min_df) + min_df for i in df]
    denormed_df = pd.DataFrame(denormed)
    denormed_df.columns = [colname]
    return denormed_df

# Construction of  LSTM neural network 

class LSTM(nn.Module):
    #init 
    def __init__(self,hidden_size,num_classes, num_layers, input_size):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,num_classes)

    #passing data 
    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        return out

#Generating lag sequence 
class SequenceDataset(Dataset):
    #init
    def __init__(self, df, sequence_length,label,features):
        self.df = df
        self.sequence_length = sequence_length
        self.label = label
        self.features = features
        self.y = np.asarray(df[label]) # convert label to np array
        self.X = np.asarray(df[features]) # convert features to np array
    # determine length of X
    def __len__(self):
        return self.X.shape[0]
    # Take data with length sequence_length as input and output 1 time-step
    def __getitem__(self):
        x=[]
        y=[]
        for i in range(sequence_length+1 , len(df)-2-sequence_length):
            x.append(np.array(df.iloc[i-1-sequence_length:i]))
            y.append(np.array(df.iloc[i]))
        return x,y

# Define objective of Optuna
def objective(trial):
    params = {
        "lr": trial.suggest_loguniform('lr', 1e-6, 1e-2),
        "optimizer": trial.suggest_categorical('optimizer',["SGD", "Adam","Adagrad"]),
        "hidden_size": trial.suggest_int('hidden_size', 8, 128, log = True)
    }
    model = LSTM(
                 params['hidden_size'],
                 num_classes =1,
                 num_layers =1,
                 input_size=1)
    model = model.to(device)
    loss_fn = torch.nn.MSELoss()
    optimizer = getattr(
        torch.optim,
        params["optimizer"]
    )\
        (
            model.parameters(),
            lr=params["lr"]
        )
    model, loss,accuracy = trainLSTM(trial,model, loss_fn,optimizer,num_epochs = 4000)
    trial.set_user_attr(key="best_booster", value=model)
    return accuracy

#Define train function
def trainLSTM(trial,model, loss_fn,optimizer,num_epochs = 4000):
    running_loss = 0.0
    epoch_steps = 0
    X_train, y_train, X_val,y_val, X_test, y_test, Xtest_tensor, ytest_tensor,max_df,min_df = loaddata()
    for epoch in range(num_epochs):
        outputs = model(X_train)
        optimizer.zero_grad()
        loss = loss_fn(outputs, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        epoch_steps += 1
        if epoch % 100 == 0:
            print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()/epoch_steps))
            running_loss = 0.00000
    # Validate
        model.eval()
        val_loss = 0.0
        val_steps = 0
    # Test validation data
        with torch.no_grad():
            inputs = X_val.to(device)
            labels = y_val.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            outputs1 = outputs.cpu().numpy()
            labels1 = labels.cpu().numpy()
            val_loss += loss.cpu().numpy()
            val_steps +=1
    return (model, val_loss / val_steps, r2_score(labels1,outputs1))
    print("Finished Training")

#Define predict function
def predict(device,model,max_df,min_df,Xtest_tensor,ytest_tensor):
    model.eval()
    with torch.no_grad():
        inputs = Xtest_tensor.to(device)
        labels = ytest_tensor.to(device)
        outputs= model(inputs)
        data_predict = outputs.cpu().numpy()
        ytest_plot = labels.cpu().numpy()
    ypred = denormalize(data_predict, max_df, min_df, "close")
    ytrue = denormalize(ytest_plot, max_df, min_df, "close")
    print(ypred)
    #Plot predicted against real data
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=ypred['close'], name='Predicted', mode='markers'))
    fig.add_trace(go.Scatter(y=ytrue['close'], name="True", mode='markers'))
    fig.update_xaxes(title='Time')
    fig.update_yaxes(title='Price')
    fig.update_layout(title_text="Intraday price of EUR/USD")
    fig.show()
    print(r2_score(ytrue['close'], ypred['close'])) # calculate R squared
    return r2_score(ytrue['close'], ypred['close'])

# Set callback
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_booster", value=trial.user_attrs["best_booster"])

#Load data and create train,val, test datasets
def loaddata():
    #Load data to pandas
    df = pd.read_csv("C:/Temp/EURUSD1646220413.029471.csv",
                                header = 0,
                                parse_dates= True,
                                infer_datetime_format=True)
    df['timestamp']= pd.to_numeric(df['timestamp'].astype('datetime64[ns]').sort_values()).sort_values
    print(df)
    dff = df.iloc[:,4] #subset last column ("close")
    print(dff)
    max_df = dff.max() #max of feature column
    min_df = dff.min() #min of feature column
    train = dff.iloc[:int(len(dff)*0.8)] #train set (0.8)
    test = dff.iloc[int(len(dff)*0.8):]#test set (0.2)
    train_df = normalize(train, max_df,min_df,"close") # normalize train and test sets
    test_df = normalize(test, max_df,min_df,"close")
    print(train_df)
    train_subset = train_df.iloc[:int(len(train_df)*0.8)] #subset train set into train_subset (ratio 0.8) and val_subset (ratio 0.2)
    val_subset = train_df.iloc[int(len(train_df)*0.8):]
    print(train_subset)
    print(val_subset)
    # Generate train, val, test datasets using SequenceDataset class
    train_dataset = SequenceDataset(train_subset,
                                                                sequence_length= 2,
                                                                label= label,
                                                                features = features
                                                                )
    val_dataset = SequenceDataset(val_subset,
                                                                sequence_length= 2,
                                                                label= label,
                                                                features = features
                                                                )
    test_dataset = SequenceDataset(test_df,
                                                            sequence_length= 1,
                                                            label = label,
                                                            features = features,
                                                              )
    
    #Reshape and convert X,y to numpy array . Shape of X(batch,sequence_length,num_feature)
    train_dataset.X = np.asarray(train_dataset.X).reshape(-1,2,1)
    train_dataset.y = np.asarray(train_dataset.y).reshape(-1,1)
    val_dataset.X = np.asarray(val_dataset.X).reshape(-1,2,1)
    val_dataset.y = np.asarray(val_dataset.y).reshape(-1,1)
    test_dataset.X = np.asarray(test_dataset.X).reshape(-1,1,1)
    test_dataset.y = np.asarray(test_dataset.y).reshape(-1,1)
    # Convert X,y to tensor
    Xtrain_tensor = torch.Tensor(train_dataset.X)
    ytrain_tensor = torch.Tensor(train_dataset.y)
    Xval_tensor = torch.Tensor(val_dataset.X)
    yval_tensor = torch.Tensor(val_dataset.y)
    Xtest_tensor = torch.Tensor(test_dataset.X)
    ytest_tensor = torch.Tensor(test_dataset.y)
    #Load X,y in batches using DataLoader
    Xtrain_dataloader = DataLoader(Xtrain_tensor,batch_size = 4, shuffle = False, drop_last=True)
    ytrain_dataloader = DataLoader(ytrain_tensor, batch_size = 4, shuffle = False, drop_last = True)
    Xval_dataloader = DataLoader(Xval_tensor,batch_size = 4, shuffle = False, drop_last=True)
    yval_dataloader = DataLoader(yval_tensor, batch_size = 4, shuffle = False, drop_last = True)
    Xtest_dataloader = DataLoader(Xtest_tensor,batch_size = 4, shuffle = False, drop_last=True)
    ytest_dataloader = DataLoader(ytest_tensor, batch_size = 4, shuffle = False, drop_last = True)
    # iterate over all dataset
    X_train = next(iter(Xtrain_dataloader))
    y_train = next(iter(ytrain_dataloader))
    X_val = next(iter(Xval_dataloader))
    y_val = next(iter(yval_dataloader))
    X_test = next(iter(Xtest_dataloader))
    y_test = next(iter(ytest_dataloader))
    return X_train, y_train, X_val,y_val, X_test, y_test, Xtest_tensor, ytest_tensor,max_df,min_df
# main function
def main(num_samples = 8):
    best_booster = None
    _, _, _,_, _, _, Xtest_tensor, ytest_tensor,max_df,min_df = loaddata()
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study\
            (sampler=sampler,
            pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=5, interval_steps=3),
                                                                                  direction='maximize')
    study.optimize(func=objective, n_trials=40,callbacks=[callback])
    model = study.user_attrs["best_booster"]
    predict(device,model,max_df,min_df,Xtest_tensor,ytest_tensor)

if __name__ == "__main__":
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    main()






[I 2022-05-16 08:39:37,808] A new study created in memory with name: no-name-fe31ef1c-6c48-49bb-becf-a081a0fbbff4


                                             timestamp     open     high  \
0    <bound method Series.sort_values of 985    164...  1.10993  1.10998   
1    <bound method Series.sort_values of 985    164...  1.10904  1.11002   
2    <bound method Series.sort_values of 985    164...  1.10993  1.10993   
3    <bound method Series.sort_values of 985    164...  1.10995  1.11008   
4    <bound method Series.sort_values of 985    164...  1.10930  1.11031   
..                                                 ...      ...      ...   
981  <bound method Series.sort_values of 985    164...  1.11954  1.11979   
982  <bound method Series.sort_values of 985    164...  1.11960  1.12005   
983  <bound method Series.sort_values of 985    164...  1.11948  1.11975   
984  <bound method Series.sort_values of 985    164...  1.12002  1.12057   
985  <bound method Series.sort_values of 985    164...  1.12015  1.12034   

        low    close  
0    1.1095  1.10973  
1    1.1088  1.10995  
2    1.1087  1.109

[I 2022-05-16 08:39:44,467] Trial 0 finished with value: -635.5669969560087 and parameters: {'lr': 6.222507929095363e-06, 'optimizer': 'Adam', 'hidden_size': 10}. Best is trial 0 with value: -635.5669969560087.


                                             timestamp     open     high  \
0    <bound method Series.sort_values of 985    164...  1.10993  1.10998   
1    <bound method Series.sort_values of 985    164...  1.10904  1.11002   
2    <bound method Series.sort_values of 985    164...  1.10993  1.10993   
3    <bound method Series.sort_values of 985    164...  1.10995  1.11008   
4    <bound method Series.sort_values of 985    164...  1.10930  1.11031   
..                                                 ...      ...      ...   
981  <bound method Series.sort_values of 985    164...  1.11954  1.11979   
982  <bound method Series.sort_values of 985    164...  1.11960  1.12005   
983  <bound method Series.sort_values of 985    164...  1.11948  1.11975   
984  <bound method Series.sort_values of 985    164...  1.12002  1.12057   
985  <bound method Series.sort_values of 985    164...  1.12015  1.12034   

        low    close  
0    1.1095  1.10973  
1    1.1088  1.10995  
2    1.1087  1.109

[I 2022-05-16 08:39:51,412] Trial 1 finished with value: -723.8752696560148 and parameters: {'lr': 0.0034335525382882507, 'optimizer': 'Adagrad', 'hidden_size': 59}. Best is trial 0 with value: -635.5669969560087.


...

[I 2022-05-16 08:45:19,981] Trial 38 finished with value: -4.941270330896565 and parameters: {'lr': 0.000523905462291131, 'optimizer': 'Adam', 'hidden_size': 32}. Best is trial 16 with value: -0.7057920340968189.


                                             timestamp     open     high  \
0    <bound method Series.sort_values of 985    164...  1.10993  1.10998   
1    <bound method Series.sort_values of 985    164...  1.10904  1.11002   
2    <bound method Series.sort_values of 985    164...  1.10993  1.10993   
3    <bound method Series.sort_values of 985    164...  1.10995  1.11008   
4    <bound method Series.sort_values of 985    164...  1.10930  1.11031   
..                                                 ...      ...      ...   
981  <bound method Series.sort_values of 985    164...  1.11954  1.11979   
982  <bound method Series.sort_values of 985    164...  1.11960  1.12005   
983  <bound method Series.sort_values of 985    164...  1.11948  1.11975   
984  <bound method Series.sort_values of 985    164...  1.12002  1.12057   
985  <bound method Series.sort_values of 985    164...  1.12015  1.12034   

        low    close  
0    1.1095  1.10973  
1    1.1088  1.10995  
2    1.1087  1.109

[I 2022-05-16 08:45:26,799] Trial 39 finished with value: -431.05667296564593 and parameters: {'lr': 0.0030050101563023016, 'optimizer': 'Adagrad', 'hidden_size': 47}. Best is trial 16 with value: -0.7057920340968189.


        close
0    1.129841
1    1.130365
2    1.129815
3    1.130756
4    1.131207
..        ...
193  1.123934
194  1.123750
195  1.123892
196  1.123651
197  1.124090

[198 rows x 1 columns]


-3.1251274536992666
